In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score

In [3]:
#True: use example from the kaggle page

#example = True
#example = False

In [4]:
#df_actual_raw = pd.read_csv('name_data')
df_preds_raw = pd.read_csv("../Temp/df_proc_train_set.csv")

In [5]:
df_preds0 = pd.DataFrame(df_preds_raw['smiles'])
df_preds0 = pd.concat([df_preds0, df_preds_raw['sol_category']], axis=1)
df_preds0

,smiles,sol_category
0,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0.0
1,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0.0
2,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0.0
3,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0.0
4,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0.0
...,...,...
70706,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2.0
70707,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2.0
70708,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2.0
70709,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2.0


In [6]:
# list to ad
list_2add = pd.Series([1, 2, 30, 43, 50, 89, 104, 2043, 70707, 70709])

In [7]:
df_preds = pd.DataFrame()
row_list = []
#for i in list:
for value in list_2add:
    #print(df_preds0.loc[value,'sol_category'])
    to_add = int(df_preds0.loc[value,'sol_category'])
    #print(to_add)
    row_list.append(to_add)
#df_preds = pd.concat([df_preds, row[value]], axis=1)
row_list

[0, 0, 0, 0, 0, 0, 0, 1, 2, 2]

In [8]:
# add sulubility values, column 'sol_category'
#if example:
#actuals = np.array([4, 4, 3, 4, 4, 4, 1, 1, 2, 1])
#preds   = np.array([0, 2, 1, 0, 0, 0, 1, 1, 2, 1])

# random numbers
actuals = np.array([0, 2, 0, 2, 2, 1, 2, 1, 2, 2])
preds = np.array(row_list)
#else:
#actuals = np.array(df_actual['sol_category'])
#preds = np.array(df_preds['sol_category'])
print(f'{actuals} (actual values)')
print(f'{preds} (predicted values)')

[0 2 0 2 2 1 2 1 2 2] (actual values)
[0 0 0 0 0 0 0 1 2 2] (predicted values)


In [9]:
actuals.shape

(10,)

In [10]:
# following code from https://www.kaggle.com/code/aroraaman/quadratic-kappa-metric-explained-in-5-simple-steps/notebook#Create-our-own-Quadratic-Weighted-Kappa-Metric

In [11]:
def quadratic_kappa(actuals, preds, N=3):
    """This function calculates the Quadratic Kappa Metric used for Evaluation in the PetFinder competition
    at Kaggle. It returns the Quadratic Weighted Kappa metric score between the actual and the predicted values 
    of adoption rating."""
    w = np.zeros((N,N))
    O = confusion_matrix(actuals, preds)
    for i in range(len(w)): 
        for j in range(len(w)):
            w[i][j] = float(((i-j)**2)/(N-1)**2)

    act_hist=np.zeros([N])
    for item in actuals: 
        act_hist[item]+=1
    
    pred_hist=np.zeros([N])
    for item in preds: 
        pred_hist[item]+=1

    E = np.outer(act_hist, pred_hist);
    E = E/E.sum();
    O = O/O.sum();
    
    num=0
    den=0
    for i in range(len(w)):
        for j in range(len(w)):
            num+=w[i][j]*O[i][j]
            den+=w[i][j]*E[i][j]
    return (1 - (num/den))

In [12]:
def original_kappa(actuals, preds, N=3):
    """This function calculates the Quadratic Kappa Metric used for Evaluation in the PetFinder competition
    at Kaggle. It returns the Quadratic Weighted Kappa metric score between the actual and the predicted values 
    of adoption rating."""
    w = np.zeros((N,N))
    O = confusion_matrix(actuals, preds)
    for i in range(len(w)): 
        for j in range(len(w)):
            if i == j:
                w[i][j] = 0
            else:
                w[i][j] = 1

    act_hist=np.zeros([N])
    for item in actuals: 
        act_hist[item]+=1
    
    pred_hist=np.zeros([N])
    for item in preds: 
        pred_hist[item]+=1

    E = np.outer(act_hist, pred_hist);
    E = E/E.sum();
    O = O/O.sum();
    
    num=0
    den=0
    for i in range(len(w)):
        for j in range(len(w)):
            num+=w[i][j]*O[i][j]
            den+=w[i][j]*E[i][j]
    return (1 - (num/den))

In [13]:
sk_quad_kappa = cohen_kappa_score(actuals, preds, weights='quadratic')
print(f'quadratic kappa: {quadratic_kappa(actuals, preds)} (own formula)')
print(f'quadratic kappa: {sk_quad_kappa} (sklearn formula)')

quadratic kappa: 0.19047619047619047 (own formula)
quadratic kappa: 0.19047619047619047 (sklearn formula)


In [15]:
sk_cohen_kappa = cohen_kappa_score(actuals, preds, weights=None)
print(f'cohen kappa: {original_kappa(actuals, preds)} (own formula)')
print(f'cohen kappa: {sk_cohen_kappa} (sklearn formula)')

cohen kappa: 0.3055555555555556 (own formula)
cohen kappa: 0.3055555555555557 (sklearn formula)
